# Lockbox Dataset Conversion

Dataset from Reiske et al., 2025¹, containing video and pose files of individual mice solving mechanical puzzle "lockboxes" recorded from three camera perspectives (top, front, side).

- Full dataset: https://doi.org/10.14279/depositonce-23850
- Subset of dataset (used here): https://www.dropbox.com/scl/fo/h7nkai8574h23qfq9m1b2/AP4gNZOpDJJ7z0yGtbWQiOc?rlkey=w36jzxqjkghg0j0xva5zsxy2v&st=5r9msqjw&dl=0

---

¹ Reiske, P., Boon, M. N., Andresen, N., Traverso, S., Hohlbaum, K., Lewejohann, L., Thöne-Reineke, C., Hellwich, O., & Sprekeler, H. (2025). Mouse Lockbox Dataset: Behavior Recognition for Mice Solving Lockboxes (arXiv:2505.15408). arXiv. https://doi.org/10.48550/arXiv.2505.15408


<img src="assets/lockbox1.png" width="700">

From Fig. 1 in Reiske et al., 2025¹


In [ ]:
import os 
import xarray as xr
import requests
import zipfile
from pathlib import Path
import pandas as pd
from movement.kinematics import compute_velocity, compute_speed
from movement.utils.vector import compute_norm
from movement.filtering import filter_by_confidence
from movement.io import load_poses, save_poses

from ethograph import TrialTree, set_media_attrs, minimal_basics, get_project_root



def download_and_extract(url: str, data_folder: Path) -> None:
    zip_path = data_folder / "dataset.zip"
    data_folder.mkdir(parents=True, exist_ok=True)
    
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    with open(zip_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_folder)
    
    zip_path.unlink()


data_folder = get_project_root() / "data" / "lockbox"
os.makedirs(data_folder, exist_ok=True)
url = "https://www.dropbox.com/scl/fo/h7nkai8574h23qfq9m1b2/AP4gNZOpDJJ7z0yGtbWQiOc?rlkey=w36jzxqjkghg0j0xva5zsxy2v&e=1&st=5r9msqjw&dl=1"

download_and_extract(url, data_folder)

data_folder = data_folder / "labeled"

In [ ]:


fps = 30

trials = [
    "2021-02-15_07-32-44_segment1",
    "2021-03-05_08-36-42_segment1",
    "2021-05-24_07-36-05_segment1",
    "2021-05-25_08-19-50_segment2",
    "2021-05-31_07-34-21_segment2", 
    "2021-05-31_07-34-21_segment3",
    
]

ds_list = []
for trial in trials:
    files = list(data_folder.glob(f"{trial}*"))
    dlc_files = [f for f in files if f.suffix == '.h5']
    # https://ethograph.readthedocs.io/en/latest/troubleshooting/ -> ideally replace .avi with .mp4
    cam_names = [f.name for f in files if f.suffix == '.mp4'] # .avi
    dlc_names = []
    
    trial_datasets = {}
    
    for file in dlc_files:
        df = pd.read_hdf(file)
        ds = load_poses.from_dlc_style_df(df, fps=fps)
        
        csv_path = str(file).replace('.h5', '.csv')
        save_poses.to_dlc_file(ds, csv_path)
        dlc_names.append(Path(csv_path).name.replace('.csv', '_individual_0.csv'))
        
        # A bit confusing to work with features that differ depending on the camera 
        # position. Ideally replace with 3D triangulated data, 
        # see Methods in https://doi.org/10.1101/2024.07.29.605658 
        if "front-view" in str(file):    
            
            ds["front_velocity"] = compute_velocity(ds.position)
            ds["front_speed"] = compute_speed(ds.position)
            trial_datasets["front"] = ds
            
        elif "top-down-view" in str(file):
            head_centre_pos = ds.position.sel(
                keypoints=["ear_left", "ear_right"]
            ).mean("keypoints")

            topview_pos = ds["position"]
            
            ds["topview_distance_head_lever_tip"] = compute_norm(
                ds.position.sel(keypoints='lever_tip') - head_centre_pos
            )
            ds["topview_distance_head_stick_head"] = compute_norm(
                ds.position.sel(keypoints='stick_head') - head_centre_pos
            )
            ds["topview_distance_head_ball"] = compute_norm(
                ds.position.sel(keypoints='ball') - head_centre_pos
            )
            trial_datasets["top"] = ds
    
    ds_merged = xr.merge(trial_datasets.values(), compat='override')
    
    
    ds_merged["position"] = topview_pos 
    
    for var in ds_merged.data_vars:
        ds_merged[var].attrs["type"] = "features"
    
    ds_merged.attrs["trial"] = trial
    ds_merged = set_media_attrs(ds_merged, cameras=cam_names, pose=dlc_names)
    ds_list.append(ds_merged)

dt = TrialTree.from_datasets(ds_list)
dt.save(data_folder / "lockbox.nc")

Extracted type_vars_dict: {'individuals': array(['individual_0'], dtype='<U12'), 'cameras': array(['2021-02-15_07-32-44_segment1_mouse324_ball_front-view.avi',
       '2021-02-15_07-32-44_segment1_mouse324_ball_side-view.avi',
       '2021-02-15_07-32-44_segment1_mouse324_ball_top-down-view.avi'],
      dtype='<U60'), 'pose': array(['2021-02-15_07-32-44_segment1_mouse324_ball_front-view-tracks_individual_0.csv',
       '2021-02-15_07-32-44_segment1_mouse324_ball_side-view-tracks_individual_0.csv',
       '2021-02-15_07-32-44_segment1_mouse324_ball_top-down-view-tracks_individual_0.csv'],
      dtype='<U80'), 'features': ['position', 'confidence', 'front_velocity', 'front_speed', 'topview_distance_head_lever_tip', 'topview_distance_head_stick_head', 'topview_distance_head_ball'], 'space': array(['x', 'y'], dtype='<U1'), 'keypoints': array(['nose', 'ear_left', 'ear_right', 'front_paw_left',
       'front_paw_right', 'back_paw_left', 'back_paw_right', 'tail_base',
       'lever_tip', 'oth